<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/pandas_100knocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データサイエンス100本ノック（構造化データ加工編） - Python
for Google Colab

### 【注意】オリジナル版との変更点
1. Google Colab ではDockerを使えないので、2020.06.18時点の 100knocks-preprocess/docker/work/data にあるCSVファイルをPostgreSQLから入手する代わりに使います。
2. オリジナルのCSVデータ, geocode.csvの'latitude'列名の初めにスペースが入っていたため、それを削除しました。

   オリジナル(100knocks-preprocess ver.1.0): ' latitude' --> 'latitude'
   

4. オリジナルの解答を元に、必要のないlibraryはimportせず、そして必要なlibraryをAzureNotebookでインストールするように最初のセルを改変してあります。
5. また、SQLではなく上記CSVからデータを読み込むように'はじめに'の最初のセルを改変してあります。

### はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートと~~データベース（PostgreSQL）~~ 100knocks-preprocess/docker/work/data にあるCSVファイルからのデータ読み込みを行います。geocode.csvに変更を加えたため、またgit cloneをするとColabの簡便さを損なうため、noguhiro2002のgithubレポジトリから直接読み込みます。
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能)
- オリジナルの解答例を元に、必要なライブラリーをpipでインストールします。
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [ ]:
#  pip install --upgrade pip

In [ ]:
# pip install imblearn

In [ ]:
# pip install --upgrade scikit-learn

In [ ]:
# pipでオリジナルの解答に必要なライブラリーをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
df_customer =pd.read_csv('customer.csv')
df_category = pd.read_csv('category.csv')
df_product = pd.read_csv('product.csv')
df_receipt  =pd.read_csv('receipt.csv')
df_store = pd.read_csv('store.csv')
df_geocode = pd.read_csv('geocode.csv')

In [ ]:
df_geocode.head()

# 演習問題

> P-001: レシート明細のデータフレーム（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
df_receipt.head(10)

---
> P-002: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].head(10)

---
> P-003: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示させよ。ただし、sales_ymdはsales_dateに項目名を変更しながら抽出すること。

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']].\
rename(columns={'sales_ymd':'sales_date'}).head(10)

In [ ]:
df_receipt.head(3)

---
> P-004: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
.query('customer_id=="CS018205000001"')

---
> P-005: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
.query('customer_id=="CS018205000001"&amount>=1000')

---
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','quantity','amount']]\
.query('customer_id=="CS018205000001"&amount>=1000|quantity>=5')

---
> P-007: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
.query('customer_id=="CS018205000001"& 1000 <= amount <= 2000')

---
> P-008: レシート明細のデータフレーム（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
df_receipt[['sales_ymd','customer_id','product_cd','amount']]\
.query('customer_id=="CS018205000001"& product_cd != "P071401019"')

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。

`df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
df_store.query('prefecture_cd != "13" & floor_area <=900')

---
> P-010: 店舗データフレーム（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_store.query("store_cd.str.startswith('S14')",engine='python').head(10)

---
> P-011: 顧客データフレーム（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query("customer_id.str.endswith('1')",engine='python').head(10)

---
> P-012: 店舗データフレーム（df_store）から横浜市の店舗だけ全項目表示せよ。

In [ ]:
df_store.head(3)

In [ ]:
df_store.query("address.str.contains('横浜市')",engine='python')

---
> P-013: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query("status_cd.str.contains('^[A-F]')",engine='python').head(10)
# df_customer.query("status_cd.str.contains('^A|^B|^C|^D|^E|^F')", engine='python').head(10)

In [ ]:
#for文での回答
df_customer[[item[0] in set('ABCDEF')
              for item in df_customer['status_cd'].tolist()]][:10]

---
> P-014: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query("status_cd.str.contains('[1-9]$')", engine='python').head(10)

---
> P-015: 顧客データフレーム（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件だけ表示せよ。

In [ ]:
df_customer.query("status_cd.str.contains('^[A-F].*[1-9]$')",engine='python').head(10)

---
> P-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
df_store.query("tel_no.str.contains('[0-9]{3}-[0-9]{3}-[0-9]{4}')",engine='python')

---
> P-17: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.head(3)

In [ ]:
df_customer.sort_values('birth_day',ascending=True).head(10)

---
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [ ]:
df_customer.sort_values('birth_day',ascending=False).head(10)

---
> P-19: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
df_receipt.head(3)

In [ ]:
df_tmp=pd.concat([df_receipt[['customer_id','amount']],
                 df_receipt['amount'].rank(method='min',ascending=False)],axis=1)
df_tmp.columns=['customer_id','amount','ranking']
df_tmp.sort_values('ranking',ascending=True).head(10)                  


In [ ]:
#Tosakas_ans
df_amount_rank = pd.concat([df_receipt[['customer_id', 'amount']] \
                            ,df_receipt['amount'].rank(method='min', ascending=False)], axis=1)
df_amount_rank.columns = ['customer_id', 'amount', 'amount_ranking']
df_amount_rank.sort_values('amount_ranking', ascending=True).head(10)

---
> P-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
df_tmp = pd.concat([df_receipt[['customer_id','amount']],\
                   df_receipt['amount'].rank(method='first',ascending=False)],axis=1)
df_tmp.columns = ['customer_id','amount','ranking']
df_tmp.sort_values('ranking',ascending=True).head(10)

In [ ]:
#Tosakas_ans
df_amount_rank= pd.concat([df_receipt[['customer_id','amount']]\
                           ,df_receipt['amount'].rank(method='first',ascending=False)],axis=1)
df_amount_rank.columns = ['customer_id','amount','amount_ranking']
df_amount_rank.sort_values('amount_ranking',ascending=True).head(10)

---
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [ ]:
len(df_receipt)

---
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
len(df_receipt['customer_id'].unique())

---
> P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
df_receipt.head(3)

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':'sum','quantity':'sum'}).reset_index()

---
> P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head(10)

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'max'}).reset_index().head(10)

---
> P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [ ]:
df_receipt.groupby('customer_id').sales_ymd.min().reset_index().head(10)

In [ ]:
df_receipt.groupby('customer_id').agg({'sales_ymd':'min'}).reset_index().head()

---
> P-026: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）と古い売上日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
#tosakas ans
df_sales_ymd = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_sales_ymd.columns = ['customer_id','sales_ymd_max','sales_ymd_min']
df_sales_ymd.query('sales_ymd_max != sales_ymd_min').head(10)

In [ ]:
df_tmp = df_receipt.groupby('customer_id').agg({'sales_ymd':['max','min']}).reset_index()
df_tmp.columns = ["_".join(pair) for pair in df_tmp.columns]
df_tmp.query('sales_ymd_max != sales_ymd_min').head(10)

---
> P-027: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.mean().reset_index().sort_values('amount',ascending=False).head(5)

In [ ]:
#tosakas ans
df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index().sort_values('amount',ascending=False).head(5)

---
> P-028: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.median().reset_index().sort_values('amount',ascending=False).head(5)

In [ ]:
df_receipt.groupby('store_cd').agg({'amount':'median'}).reset_index().sort_values('amount',ascending=False).head(5)

---
> P-029: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求めよ。

In [ ]:
df_receipt.groupby('store_cd').product_cd.apply(lambda x : x.mode()).reset_index()

In [ ]:
#tosakas ans
df_receipt.groupby('store_cd').product_cd.apply(lambda x : x.mode()).reset_index()\
.set_index(['store_cd','level_1','product_cd'])

---
> P-030: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本分散を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.var(ddof=0).reset_index().sort_values('amount',ascending=False).head(5)

In [ ]:
#aggを用いた別解
df_receipt.groupby('store_cd').agg({'amount':'var'}).reset_index().sort_values('amount',ascending=False).head(5)

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.std().reset_index().sort_values('amount',ascending=False).head(5)

In [ ]:
#aggを用いた別解
df_receipt.groupby('store_cd').agg({'amount':'std'}).reset_index().sort_values('amount',ascending=False).head(5)

---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
#コード例1
np.percentile(df_receipt['amount'],q=[25,50,75,100])

In [ ]:
#コード例2
df_receipt.amount.quantile(q=np.arange(5)/4)

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
df_receipt.groupby('store_cd').amount.mean().reset_index().query('amount>=330')

In [ ]:
#aggを用いた別解
df_receipt.groupby('store_cd').agg({'amount':'mean'}).reset_index().query('amount>=330')

---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [ ]:
#queryを使わない書き方
df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().mean()

In [ ]:
# queryを使う書き方
df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').amount.sum().mean()

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [ ]:
df_receipt.head()

In [ ]:
amount_mean =df_receipt[~df_receipt['customer_id'].str.startswith('Z')].groupby('customer_id').amount.sum().mean()
df_amount_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_amount_sum[df_amount_sum['amount'] >= amount_mean].head(10)

---
> P-036: レシート明細データフレーム（df_receipt）と店舗データフレーム（df_store）を内部結合し、レシート明細データフレームの全項目と店舗データフレームの店舗名（store_name）を10件表示させよ。

In [ ]:
df_receipt.head(1)

In [ ]:
df_store.head(1)

In [ ]:
pd.merge(df_receipt,df_store[['store_cd','store_name']],how='inner',on='store_cd').head(10)

---
> P-037: 商品データフレーム（df_product）とカテゴリデータフレーム（df_category）を内部結合し、商品データフレームの全項目とカテゴリデータフレームの小区分名（category_small_name）を10件表示させよ。

In [ ]:
df_product.head(1)

In [ ]:
df_category.head(1)

In [ ]:
pd.merge(df_product,
         df_category[['category_major_cd','category_medium_cd','category_small_cd','category_small_name']],
         how='inner',on=['category_major_cd','category_medium_cd','category_small_cd']).head(10)

---
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は10件だけ表示させれば良い。

In [ ]:
df_customer.head(1)

In [ ]:
df_receipt.head(10)

In [ ]:
df_amount_sum= df_receipt.groupby('customer_id').amount.sum().reset_index()
df_tmp = df_customer.query('gender_cd == "1" and not customer_id.str.startswith("Z")',engine='python')
pd.merge(df_tmp['customer_id'],df_amount_sum,how='left',on='customer_id').fillna(0).head(10)

---
> P-039: レシート明細データフレーム（df_receipt）から売上日数の多い顧客の上位20件と、売上金額合計の多い顧客の上位20件を抽出し、完全外部結合せよ。ただし、非会員（顧客IDが'Z'から始まるもの）は除外すること。

In [ ]:
df_sum = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_sum = df_sum.query('not customer_id.str.startswith("Z")',engine='python')
df_sum = df_sum.sort_values('amount',ascending=False).head(20)

df_cnt = df_receipt[~df_receipt.duplicated(subset=['customer_id','sales_ymd'])]
df_cnt = df_cnt.query('not customer_id.str.startswith("Z")',engine='python')
df_cnt = df_cnt.groupby('customer_id').sales_ymd.count().reset_index()
df_cnt = df_cnt.sort_values('sales_ymd',ascending=False).head(20)

pd.merge(df_sum,df_cnt,how='outer',on='customer_id')

---
> P-040: 全ての店舗と全ての商品を組み合わせると何件のデータとなるか調査したい。店舗（df_store）と商品（df_product）を直積した件数を計算せよ。

In [ ]:
df_store.head(3)

In [ ]:
df_product.head(3)

In [ ]:
df_store_tmp = df_store.copy()
df_product_tmp = df_product.copy()
df_store_tmp['key'] = 0
df_product_tmp['key'] = 0
len(pd.merge(df_store_tmp,df_product_tmp,how='outer',on='key'))

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [ ]:
df_receipt.head(3)

In [ ]:
df_sales_amount_by_date = df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum().reset_index()
df_sales_amount_by_date = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift()],axis=1)
df_sales_amount_by_date.columns={'sales_ymd','amount','lag_ymd','lag_amount'}
df_sales_amount_by_date['diff_amount'] = df_sales_amount_by_date['amount']- df_sales_amount_by_date['lag_amount']
df_sales_amount_by_date.head(10)

---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [ ]:
# （縦持ちのケース）
df_sales_amount_by_date = df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum().reset_index()
for i in range(1,4):
    if i == 1:
        df_lag = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(i)],axis=1)
    else:
        df_lag = df_lag.append(pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(i)],axis=1))
df_lag.columns=['sales_ymd','amount','lag_sales_ymd','lag_sales_amount']
df_lag.dropna().sort_values(['sales_ymd','lag_sales_ymd']).head(10)        

In [ ]:
# 横持ちのケース
df_sales_amount_by_date = df_receipt[['sales_ymd','amount']].groupby('sales_ymd').sum().reset_index()
for i in range(1,4):
    if i == 1:
        df_lag = pd.concat([df_sales_amount_by_date,df_sales_amount_by_date.shift(i)],axis=1)
    else:
        df_lag = pd.concat([df_lag,df_sales_amount_by_date.shift(i)],axis=1)
        
df_lag.columns = ['sales_ymd','amount','lag1_sales_ymd','lag1_amount',
                 'lag2_sales_ymd','lag2_amount','lag3_sales_ymd','lag3_amount']
df_lag.dropna().sort_values('sales_ymd').head(10)

---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
df_tmp = pd.merge(df_receipt,df_customer,how='inner',on='customer_id')
df_tmp['era'] = df_tmp['age'].apply(lambda x : math.floor(x/10)*10)
df_sales_summary = pd.pivot_table(df_tmp,index='era',columns='gender_cd',values='amount',aggfunc='sum').reset_index()
df_sales_summary.columns=['era','male','female','unknown']
df_sales_summary

---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [ ]:
df_sales_summary = df_sales_summary.set_index('era').\
    stack().reset_index().replace({'female':'01',
                                  'male':'00',
                                  'unknown':'99'}).rename(columns={'level_1':'gender_cd',0:'amount'})

In [ ]:
df_sales_summary

---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_customer.head(3)

In [ ]:
pd.concat([df_customer['customer_id'],
          pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y/%m/%d')],
         axis=1).head(10)

---
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_customer['customer_id'],pd.to_datetime(df_customer['application_date'])],axis=1).head(10)

---
> P-047: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_receipt.head(3)

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
           pd.to_datetime(df_receipt['sales_ymd'].astype(str))],axis=1).head(10)

---
> P-048: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型（dateやdatetime）に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_receipt.head(3)

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
          pd.to_datetime(df_receipt['sales_epoch'].astype(int),unit='s')],axis=1).head(10)

---
> P-049: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"年"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。データは10件を抽出すれば良い。

In [ ]:
df_receipt.head(3)

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
          pd.to_datetime(df_receipt['sales_ymd'],unit='s').dt.year],axis=1).head(10)

---
> P-050: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"月"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"月"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
          pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%m')],axis=1).head(10)

---
> P-051: レシート明細データフレーム（df_receipt）の売上エポック秒（sales_epoch）を日付型（timestamp型）に変換し、"日"だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに抽出せよ。なお、"日"は0埋め2桁で取り出すこと。データは10件を抽出すれば良い。

In [ ]:
pd.concat([df_receipt[['receipt_no','receipt_sub_no']],
         pd.to_datetime(df_receipt['sales_epoch'],unit='s').dt.strftime('%d')],axis=1).head(10)

---
> P-052: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2000円以下を0、2000円超を1に2値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')

df_sales_amount = df_sales_amount[['customer_id','amount']].groupby('customer_id').sum().reset_index()

df_sales_amount['sales_flg'] = df_sales_amount['amount'].apply(lambda x : 1 if x>2000 else 0)
df_sales_amount.head(10)

In [ ]:
# 解2
# np.where
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')
df_sales_amount = df_sales_amount[['customer_id','amount']].groupby('customer_id').sum().reset_index()

# np.where(条件式, x(真の場合), y(偽の場合))
df_sales_amount['sales_fig'] = np.where(df_sales_amount['amount']>2000,1,0)
df_sales_amount.head(10)

In [ ]:
# 解3
df_sales_amount = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount.sum().reset_index()
df_sales_amount.loc[df_sales_amount['amount']<=2000,'threshold'] = 0
df_sales_amount.loc[df_sales_amount['amount']>2000,'threshold'] = 1
df_sales_amount.head(10)



> P-053: 顧客データフレーム（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に２値化せよ。さらにレシート明細データフレーム（df_receipt）と結合し、全期間において買い物実績のある顧客数を、作成した2値ごとにカウントせよ。

In [ ]:
df_customer.head(3)

In [ ]:
df_receipt.head(3)

In [ ]:
# 郵便番号（postal_cd）を２値化する(東京:1, その他:0)
df_tmp = df_customer[['customer_id','postal_cd']].copy()
df_tmp['postal_flg'] = df_tmp['postal_cd'].apply(lambda x : 1 if 100 <= int(x[0:3]) <=209 else 0)

# レシート明細データフレーム（df_receipt）と結合(pd.merge(df1, df2, on='キー', how='inner'))
# ユニークな要素の個数(重複のない個数)をcustomer_idごとに算出(pandas.DataFrame.nunique())
pd.merge(df_tmp,df_receipt,how='inner',on='customer_id').groupby('postal_flg').agg({'customer_id':'nunique'})

In [ ]:
# 別解
# np.where
df_tmp = df_customer[['customer_id','postal_cd']].copy()
df_tmp['postal_flg'] = np.where(df_tmp['postal_cd'].str[0:3].astype(int).
                               between(100,209),1,0)
pd.merge(df_tmp,df_receipt,how='inner',on='customer_id').groupby('postal_flg').agg({'customer_id':'nunique'})

---
> P-054: 顧客データデータフレーム（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに抽出せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。結果は10件表示させれば良い。

In [ ]:
df_customer.head(2)

In [ ]:
# 住所（address）の都道府県部分を抽出し、それぞれ値をつける
# map()の引数に辞書dict（{key: value}）を指定すると、keyと一致する要素がvalueに置き換えられる。
pd.concat([df_customer[['customer_id','address']],
          df_customer['address'].str[0:3].map({'埼玉県':'11',
                                               '千葉県':'12',
                                               '東京都':'13',
                                               '神奈川':'14'})],axis=1).head(10)

---
> P-055: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額と合計ともに表示せよ。カテゴリ値は上から順に1〜4とする。結果は10件表示させれば良い。
>
> - 最小値以上第一四分位未満
> - 第一四分位以上第二四分位未満
> - 第二四分位以上第三四分位未満
> - 第三四分位以上

In [ ]:
#コード例1
# 顧客ID（customer_id）ごとにグループ分け(groupby)し、売上金額（amount）を合計する(sum)
df_sales_amount = df_receipt[['customer_id','amount']].groupby('customer_id').sum().reset_index()
# 売上金額（amount）ごとに四分位点を求める。(25パーセンタイル: 25pct, 50パーセンタイル: 50pct, 75パーセンタイル: 75pct)
pct25 = np.quantile(df_sales_amount['amount'],0.25)
pct50 = np.quantile(df_sales_amount['amount'],0.50)
pct75 = np.quantile(df_sales_amount['amount'],0.75)

# カテゴリ値の関数を作成し、適用する
def pct_group(x):
    if x < pct25:
        return 1
    elif pct25 <= x < pct50:
        return 2
    elif pct50 <= x < pct75:
        return 3
    elif pct75 <= x:
        return 4
# applyを用いてpct_groupを各行に適用する
df_sales_amount['pct_group'] = df_sales_amount['amount'].apply(lambda x: pct_group(x))
df_sales_amount.head(10)
    

In [ ]:
# 確認用
print('pct25:',pct25)
print('pct50:',pct50)
print('pct75:',pct75)

In [ ]:
#mayukiアレンジ
df_sales_amount = df_receipt[['customer_id','amount']].groupby('customer_id').sum().reset_index()
q1 = df_sales_amount['amount'].describe()['25%']
q2 = df_sales_amount['amount'].describe()['50%']
q3 = df_sales_amount['amount'].describe()['75%']
print('pct25:',q1)
print('pct50:',q2)
print('pct75:',q3)

In [ ]:
#コード例2
df_tmp = df_receipt.groupby('customer_id')[['amount']].sum()
df_tmp['quentile'] ,bins= pd.qcut(df_receipt.groupby('customer_id')['amount'].sum(),4,retbins=True)
display(df_tmp.head(10))
print('quantiles:',bins)

---
> P-056: 顧客データフレーム（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに抽出せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。先頭10件を表示させればよい。

In [ ]:
# コード例1
# 年齢（age）を10歳刻みで年代を算出
# math.floorで切り捨て。min(, 60)で６０以上が出力されない。
df_customer_era = pd.concat([df_customer[['customer_id','birth_day']],
                            df_customer['age'].apply(lambda x : min(math.floor(x / 10)*10,60))],axis=1)
df_customer_era.head(10)

In [ ]:
# コード例2
df_customer['age_group'] = pd.cut(df_customer['age'],bins=[1,10,20,30,40,50,60,np.inf],right=False)
df_customer[['customer_id','birth_day','age_group']].head(10)

In [ ]:
# (別解)
# 年齢（age）を(10代未満、１０代、２０代、３０代、４０代、５０代、６０代以上)で分ける。(flg_age)
def age_group(x):
    if x < 10:
        return '10代未満'
    elif 10 <= x < 20:
        return '10代'
    elif 20 <= x < 30:
        return '20代'
    elif 30 <= x < 40:
        return '30代'
    elif 40 <= x < 50:
        return '40代'
    elif 50 <= x < 60:
        return '50代'
    elif 60 <= x:
        return '60代以上'
    
df_customer['flg_age'] = df_customer['age'].apply(lambda x : age_group(int(x)))
# 顧客ID（customer_id）、生年月日（birth_day）とともに抽出
df_customer[['customer_id','birth_day','flg_age']].head(10)
   


> P-057: 前問題の抽出結果と性別（gender）を組み合わせ、新たに性別×年代の組み合わせを表すカテゴリデータを作成せよ。組み合わせを表すカテゴリの値は任意とする。先頭10件を表示させればよい。

In [ ]:
# カテゴリーデータ(性別x年代)'gender_era'を作成する
# int型なのでastype(str)で変換する
df_customer_era['era_gender'] = df_customer['gender_cd'].astype('str') + df_customer_era['age'].astype('str')
df_customer_era.head(10)

---
> P-058: 顧客データフレーム（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに抽出せよ。結果は10件表示させれば良い。

In [ ]:
# 性別コード（gender_cd）をダミー変数化する(get_dummies)
# 引数columnsにダミー化したい列の列名をリストで指定
pd.get_dummies(df_customer[['customer_id','gender_cd']],columns=['gender_cd']).head(10)

---
> P-059: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに表示せよ。標準化に使用する標準偏差は、不偏標準偏差と標本標準偏差のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
# skleanのpreprocessing.scaleを利用するため、標本標準偏差で計算されている
# 顧客IDが"Z"から始まるのものを除外し、顧客ID（customer_id）ごとに売上金額（amount）を合計する
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').agg({'amount':'sum'}).reset_index()
# 売り上げ金額を標準化する
df_sales_amount['amount_ss'] = preprocessing.scale(df_sales_amount['amount'])
df_sales_amount.head(10)
                           

In [ ]:
# (別解)
# コード例2（fitを行うことで、別のデータでも同じの平均・標準偏差で標準化を行える）
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').\
groupby('customer_id').agg({'amount':'sum'}).reset_index()
scaler=preprocessing.StandardScaler()
scaler.fit(df_sales_amount[['amount']])
df_sales_amount['amount_ss'] = scaler.transform(df_sales_amount[['amount']])
df_sales_amount.head(10)

---
> P-060: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
# skleanのpreprocessing.scaleを利用するため、標本標準偏差で計算されている
# 売上金額（amount）を顧客ID（customer_id）ごとに合計
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').agg({'amount':'sum'}).reset_index()
# 売上金額を最小値0、最大値1に正規化(preprocessing.minmax_scale)
df_sales_amount['amount_mm'] = preprocessing.minmax_scale(df_sales_amount['amount'])
df_sales_amount.head(10)

In [ ]:
# (別解)
# コード例2（fitを行うことで、別のデータでも同じの平均・標準偏差で標準化を行える）
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').agg({'amount':'sum'}).reset_index()
scaler = preprocessing.MinMaxScaler()
scaler.fit(df_sales_amount[['amount']])
df_sales_amount['amount_mm'] =scaler.transform(df_sales_amount[['amount']])
df_sales_amount.head(10)

---
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').amount.sum().reset_index()
df_sales_amount['amount_log10'] = df_amount_sum['amount'].apply(lambda x : math.log10(x+1))
df_sales_amount.head(10)

In [ ]:
# 別解
# np.log10を用いている
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_sales_amount['amount_log10'] = np.log10(df_sales_amount['amount']+1)
df_sales_amount.head(10)

---
> P-062: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を自然対数化(底=e）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。結果は10件表示させれば良い。

In [ ]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').agg({'amount':'sum'}).reset_index()
df_sales_amount['amount_log'] = np.log(df_sales_amount['amount']+1) 
df_sales_amount.head(10)

In [ ]:
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python')\
.groupby('customer_id').amount.sum().reset_index()
df_sales_amount['amount_log'] = np.log(df_sales_amount['amount']+1)
df_sales_amount.head(10)

---
> P-063: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益額を算出せよ。結果は10件表示させれば良い。

In [ ]:
df_product.head(2)

In [ ]:
df_tmp = df_product.copy()
df_tmp['unit_profit'] = df_tmp['unit_price'] - df_tmp['unit_cost']
df_tmp.head(10)


> P-064: 商品データフレーム（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。
ただし、単価と原価にはNULLが存在することに注意せよ。

In [ ]:
df_tmp = df_product.copy()
df_tmp ['unit_profit_rate'] = (df_tmp['unit_price']-df_tmp['unit_cost']) / df_tmp['unit_price']
df_tmp['unit_profit_rate'].mean(skipna=True)

---
> P-065: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
# 商品データフレーム（df_product）をコピーする。
df_tmp = df_product.copy()
# 原価（unit_cost）から利益率が30%となる新たな単価(new_price)を求める(1円未満は切り捨て)
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x :np.floor(x/0.7))
# 利益率を確認する
df_tmp['new_price_rate'] = (df_tmp['new_price']-df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

In [ ]:
# (別解)
# 商品データフレーム（df_product）をコピーする。
df_tmp = df_product.copy()
# 原価（unit_cost）から利益率が30%となる新たな単価(new_price)を求める(1円未満は切り捨て)
# 原価（unit_cost） x 10/7 (利益率が30%となる)計算式。math.floorはエラー
df_tmp['new_price'] = np.floor(df_tmp['unit_cost']*10/7)
# 利益率を確認する
df_tmp['new_price_rate'] = (df_tmp['new_price']-df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)

---
> P-067: 商品データフレーム（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）にはNULLが存在することに注意せよ。

In [ ]:
# 商品データフレーム（df_product）をコピーする。
df_tmp = df_product.copy()
# 原価（unit_cost）から利益率が30%となる新たな単価(new_price)を求める(1円未満を切り上げ)
# math.ceilはNaNでエラーとなるが、numpy.ceilはエラーとならない
df_tmp['new_price'] = df_tmp['unit_cost'].apply(lambda x :np.ceil(x/0.7))
df_tmp['new_price_rate'] = (df_tmp['new_price']-df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)


In [ ]:
# 別解
# 商品データフレーム（df_product）をコピーする。
df_tmp = df_product.copy()
# 原価（unit_cost）から利益率が30%となる新たな単価(new_price)を求める(1円未満を切り上げ)
# 原価（unit_cost） x 10/7 (利益率が30%となる)計算式。
# math.ceilはNaNでエラーとなるが、numpy.ceilはエラーとならない
df_tmp['new_price']  = np.ceil(df_tmp['unit_cost']*10/7)
# 利益率を確認する
df_tmp['new_price_rate'] =(df_tmp['new_price'] - df_tmp['unit_cost']) / df_tmp['new_price']
df_tmp.head(10)
                               

---
> P-068: 商品データフレーム（df_product）の各商品について、消費税率10%の税込み金額を求めよ。 1円未満の端数は切り捨てとし、結果は10件表示すれば良い。ただし、単価（unit_price）にはNULLが存在することに注意せよ。

In [ ]:
# 商品データフレーム（df_product）をコピーする。
df_tmp = df_product.copy()
# math.floorはNaNでエラーとなるが、numpy.floorはエラーとならない
df_tmp['price_tax'] = df_tmp['unit_price'].apply(lambda x : np.floor(x*1.1))
df_tmp.head(10)

In [ ]:
# 別解
df_tmp = df_product.copy()
df_tmp['price_tax'] = np.floor(df_tmp['unit_price']*1.1)
df_tmp.head(10)

---
> P-069: レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分"07"（瓶詰缶詰）の購入実績がある顧客のみとし、結果は10件表示させればよい。

In [ ]:
df_receipt.head(3)

In [ ]:
df_product.head(3)

In [ ]:
#解１
#  agg({カラム: 適用する関数})
# レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合(pd.merge, キーはproduct_cd)
# 顧客毎に全商品の売上金額合計を算出する
df_tmp_1 = pd.merge(df_receipt,df_product,
                   how='inner',on='product_cd').groupby('customer_id').agg({'amount':'sum'}).reset_index()

# カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計
# merge(df1, df2, on='キー', how='inner')で結合
# groupbyで顧客ごとにグルーピング。amount.sum()で売上金額合計
df_tmp_2 =pd.merge(df_receipt,df_product.query('category_major_cd=="07"'),
                   how='inner',on='product_cd').groupby('customer_id').agg({'amount':'sum'}).reset_index()

# df_tmp1とdf_tmp2を結合する。
df_tmp_3 =pd.merge(df_tmp_1,df_tmp_2,how='inner',on='customer_id')
# df_tmp_3.head(2)
df_tmp_3['rate_07'] = df_tmp_3['amount_y'] / df_tmp_3['amount_x']
df_tmp_3.head(10)

In [ ]:
#解２
df_temp= df_receipt.merge(df_product,how='left',on='product_cd').\
groupby(['customer_id','category_major_cd'])['amount'].sum().unstack()
df_temp = df_temp[df_temp[7]>0]
df_temp['sum'] = df_temp.sum(axis=1)
df_temp['07_rate'] = df_temp[7] / df_temp['sum']
df_temp.head(10)

In [ ]:
#解３
# レシート明細データフレーム（df_receipt）と商品データフレーム（df_product）を結合(pd.merge, キーはproduct_cd)
# 顧客毎に全商品の売上金額合計を算出する
df_tmp_1 = pd.merge(df_receipt,df_product,how='inner',on='product_cd').groupby('customer_id').amount.sum().reset_index()
# カテゴリ大区分（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計
# merge(df1, df2, on='キー', how='inner')で結合
# groupbyで顧客ごとにグルーピング。amount.sum()で売上金額合計
df_tmp_2 = pd.merge(df_receipt,df_product.query('category_major_cd=="07"')\
                    ,how='inner',on='product_cd').groupby('customer_id').amount.sum().reset_index()
# df_tmp_2.head() 
# df_tmp1とdf_tmp2を結合する。
df_tmp_3 = pd.merge(df_tmp_1,df_tmp_2,how='inner',on='customer_id')

# df_tmp_3.head() amound_xとamound_y生成

df_tmp_3['rate_07'] = df_tmp_3['amount_y'] / df_tmp_3['amount_x']
df_tmp_3.head(10)
                    

---
> P-070: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
# レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合する
df_tmp = pd.merge(df_receipt[['sales_ymd','customer_id']],df_customer[['application_date','customer_id']],\
                 how='inner',on='customer_id')
# 重複部分を削除する
df_tmp = df_tmp.drop_duplicates()
# pandas.to_datetime()関数を使うと、日時（日付・時間）を表した文字列の列pandas.Seriesをdatetime64[ns]型に変換できる。
# sales_ymdは数値型のためastype('str')で文字列型へ変更
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
# 会員申込日（application_date）からの売上日（sales_ymd）を引く
df_tmp['elapsed_date'] = df_tmp['sales_ymd']-df_tmp['application_date']
df_tmp.head(10)

In [ ]:
#別解
df_tmp = pd.merge(df_receipt[['sales_ymd','customer_id']],df_customer[['application_date','customer_id']],\
                  how='inner',on='customer_id')
df_tmp = df_tmp.drop_duplicates()
df_tmp['elapsed_date'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))-pd.to_datetime(df_tmp['application_date'])
df_tmp[['customer_id','sales_ymd','application_date','elapsed_date']].head(10)

In [ ]:
# relativedeltaの使い方 d0（指定日）からd1（本日）までの年月日の差を求める
d0 = date(1998,5,12)
d1 = date.today()
dy = relativedelta(d1,d0)
dy

---
> P-071: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
# pd.mergeでレシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合(キーはcustomer_id)
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],\
                  how='inner',on='customer_id')
# 重複を削除する(drop_duplicates())
df_tmp = df_tmp.drop_duplicates()
# pandas.to_datetime()関数を使うと、日時（日付・時間）を表した文字列の列pandas.Seriesをdatetime64[ns]型に変換できる。
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
# 月数を計算するrelativedelta(datetime1, datetime2) datetime1 - datetime2
df_tmp['elepsed_date'] = df_tmp[['sales_ymd','application_date']].apply(lambda x :\
                                                                       relativedelta(x[0],x[1]).years * 12 +relativedelta(x[0],x[1]).months,axis=1)

# 顧客ID（customer_id）でソートし、１０件表示する
df_tmp.sort_values('customer_id').head(10)

---
> P-072: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い。（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
# pd.mergeでレシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合(キーはcustomer_id)
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],\
                 how='inner',on='customer_id')
# 重複を削除する(drop_duplicates())
df_tmp = df_tmp.drop_duplicates()
# pandas.to_datetime()関数を使うと、日時（日付・時間）を表した文字列の列pandas.Seriesをdatetime64[ns]型に変換できる。
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])
# 経過年数を計算する。relativedelta(datetime1, datetime2).yearsで年数差(axis=1で行に沿った処理)
df_tmp['elapsed_date'] = df_tmp[['sales_ymd','application_date']].apply(lambda x:\
                                                                     relativedelta(x[0],x[1]).years,axis=1)
df_tmp.head(10)

---
> P-073: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、顧客データフレーム（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
# pd.mergeでレシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合(キーはcustomer_id)
df_tmp = pd.merge(df_receipt[['customer_id','sales_ymd']],df_customer[['customer_id','application_date']],\
                 how='inner',on='customer_id')
# 重複を削除する(drop_duplicates())
df_tmp = df_tmp.drop_duplicates()

# pandas.to_datetime()関数を使うと、日時（日付・時間）を表した文字列の列pandas.Seriesをdatetime64[ns]型に変換できる。
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
df_tmp['application_date'] = pd.to_datetime(df_tmp['application_date'])

# エポック秒による経過時間を計算(timestamp()メソッドでdatetimeオブジェクトをUNIX時間に変換)
# 注：タイムスタンプは、ナノ秒を含むunix時間（10 ** 9で除算）
df_tmp['elapsed_date'] = (df_tmp['sales_ymd'].astype(np.int64) / 10**9)-\
(df_tmp['application_date'].astype(np.int64) / 10**9)

# 1０件表示する
df_tmp.head(10)

In [ ]:
# なぜ１０の９乗で除算するのか？
nano_time = pd.to_datetime('1970-01-02')
nano_time.value
#エポック時間は1970-01-01からの経過時間なので24時間x60分x60秒=86400秒
#unix時間がナノ秒（10の９乗）であることがわかる

---
> P-074: レシート明細データフレーム（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、当該週の月曜日付とともに表示せよ。結果は10件表示させれば良い（なお、sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
# レシート明細データフレーム（df_receipt）の顧客データ(customer_id),売上日（sales_ymd）
df_tmp = df_receipt[['customer_id','sales_ymd']]
# 重複を削除する
df_tmp = df_tmp.drop_duplicates()
df_tmp['sales_ymd'] = pd.to_datetime(df_tmp['sales_ymd'].astype('str'))
# 月曜のデータを追加する。(relativedelta(days=x.weekday())月曜：０, 火曜：１・・・)
df_tmp['monday'] = df_tmp['sales_ymd'].apply(lambda x : x - relativedelta(days = x.weekday()))
# 売上日（sales_ymd）から月曜日を引いて経過日数を求める
df_tmp['elapsed_weekday'] = df_tmp['sales_ymd'] - df_tmp['monday']
df_tmp.head(10)

---
> P-075: 顧客データフレーム（df_customer）からランダムに1%のデータを抽出し、先頭から10件データを抽出せよ。

In [ ]:
# 顧客データフレーム（df_customer）より抽出する(sample)
# ランダムに1%のデータは引数frac=0.01
df_customer.sample(frac=0.01).head(10)

---
> P-076: 顧客データフレーム（df_customer）から性別（gender_cd）の割合に基づきランダムに10%のデータを層化抽出データし、性別ごとに件数を集計せよ。

In [ ]:
# sklearn.model_selection.train_test_splitを使用した例
# _(訓練データ９０％)、df_tmp(テストデータ10%)とすることでランダムに１０％を抽出する(層化抽出: 引数stratify)
_, df_tmp = train_test_split(df_customer,test_size=0.1,stratify=df_customer['gender_cd'])
# 性別（gender_cd）でグループ分けし、customer_idの件数(count)を集計
df_tmp.groupby('gender_cd').agg({'customer_id' : 'count'})

In [ ]:
# (別解)
# sampleメソッドを用いてランダムに抽出(frac=0.1で１０％データを抽出)
# 性別（gender_cd）でグループ分けし、customer_idの件数(count)を集計
df_customer.sample(frac=0.1).groupby('gender_cd').agg({'customer_id' : 'count'})

In [ ]:
df_tmp.head(10)

---
> P-077: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を平均から3σ以上離れたものとする。結果は10件表示させれば良い。

In [ ]:
# 顧客IDが"Z"から始まるものを除外する
# レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計する
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').\
groupby('customer_id').amount.sum().reset_index()
# 正規化・標準化(平均０、標準偏差１)：preprocessing.scale()関数を使う。
df_sales_amount ['amount_ss'] = preprocessing.scale(df_sales_amount['amount'])
# 外れ値を抽出する(平均(mean)の ± 3σ(std)以上離れたものを抽出する)
# queryメソッドで条件に合うものを抽出。3σ以上離れたは、絶対値(abs)を用いる
df_sales_amount.query('abs(amount_ss)>=3').head(10)

In [ ]:
# (別解)
# 顧客IDが"Z"から始まるものを除外する
# レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計する
df_sales_amount = df_receipt.query("not customer_id.str.startswith('Z')", 
                                   engine='python').groupby('customer_id').amount.sum().reset_index()

# 各顧客の売り上げ合計の平均、標準偏差を求める
mean = df_sales_amount.describe().loc['mean']
std = df_sales_amount.describe().loc['std']

# 外れ値を抽出する(平均(mean)の ± 3σ(std)以上離れたものを抽出する)
df_sales_amount[(df_sales_amount['amount'] >= int(mean) + 3*int(std)) | \
                (df_sales_amount['amount'] <= int(mean) - 3*int(std))].head(10)

---
> P-078: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第一四分位と第三四分位の差であるIQRを用いて、「第一四分位数-1.5×IQR」よりも下回るもの、または「第三四分位数+1.5×IQR」を超えるものとする。結果は10件表示させれば良い。

In [ ]:
#np.ercentileを用いた回答
# 顧客IDが"Z"から始まるものを除外する
# レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計する
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',engine='python').groupby('customer_id').agg({'amount':'sum'}).reset_index()
# 第一四分位と第三四分位の差であるIQRを求める
pct75 = np.percentile(df_sales_amount['amount'],q=75)
pct25 = np.percentile(df_sales_amount['amount'],q=25)
iqr = pct75 - pct25
# 第一四分位数-1.5×IQR(amount_low), 第三四分位数+1.5×IQR(amount_hight)をそれぞれ求める
amount_low = pct25 - (iqr *1.5)
amount_hight = pct75 + (iqr * 1.5)
# 外れ値を抽出する(@で変数名)
df_sales_amount.query('amount < @amount_low or @amount_hight <amount').head(10)

In [ ]:
# 顧客IDが"Z"から始まるものを除外する
# レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客単位に合計する
df_sales_amount = df_receipt.query('not customer_id.str.startswith("Z")',\
                                  engine='python').groupby('customer_id').amount.sum().reset_index()
# 第一四分位と第三四分位の差であるIQRを求める
pct75 = df_sales_amount['amount'].quantile(0.75)
pct25 = df_sales_amount['amount'].quantile(0.25)
iqr = pct75-pct25
# 第一四分位数-1.5×IQR(amount_low), 第三四分位数+1.5×IQR(amount_hight)をそれぞれ求める
amount_low = pct25 - (iqr * 1.5)
amount_hight = pct75 + (iqr * 1.5)
# 外れ値を抽出する(@で変数名)
df_sales_amount.query('amount > @amount_hight | amount<@amount_low').head(10)

---
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
# isnull()で要素ごとに欠損値か判定
df_product.isnull().sum()

---
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [ ]:
# 商品データフレーム（df_product）をコピーする
df_product_1 = df_product.copy()
# 欠損値削除前の件数を確認する
print('削除前:',len(df_product_1))
# 欠損値を除外（削除）するにはdropna()メソッド
df_product_1.dropna(inplace=True)
print('削除後:',len(df_product_1))


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# fillnaで欠損値を補完する。
# np.roundで偶数よせの四捨五入。np.nanmeanで欠損値の値を除いた平均値
df_product_2 = df_product.fillna({'unit_price':np.round(np.nanmean(df_product['unit_price'])), 
                                  'unit_cost':np.round(np.nanmean(df_product['unit_cost']))})

# isnull().sum()で欠損を確認する
df_product_2.isnull().sum()

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たなdf_product_3を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# 欠損値はfillnaを用いる
# np.roundで偶数よせの四捨五入。中央値np.nanmedianで補完する
df_product_3 = df_product.fillna({'unit_price':np.round(np.nanmedian(df_product['unit_price'])),\
                                 'unit_cost':np.round(np.nanmedian(df_product['unit_cost']))})
# isnull().sum()で欠損を確認する
df_product_3.isnull().sum()
                                 

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品の小区分（category_small_cd）ごとに算出した中央値で補完した新たなdf_product_4を作成せよ。なお、中央値について1円未満は四捨五入とし、0.5については偶数寄せでかまわない。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# 解1
# 各商品の小区分（category_small_cd）ごとに中央値を出す
df_tmp = df_product.groupby('category_small_cd').agg({'unit_price':'median', 'unit_cost':'median'}).reset_index()
# カラム名を変更する
df_tmp.columns = ['category_small_cd', 'median_price', 'median_cost']

# 商品データフレーム（df_product）と中央値が入ったデータフレーム(df_tmp)を結合する
df_product_4 = pd.merge(df_product, df_tmp, how='inner', on='category_small_cd')

# 単価（unit_price）の欠損値に中央値を補完する(axis=1で各行に適用)
df_product_4['unit_price'] = df_product_4[['unit_price', 'median_price']]. \
                    apply(lambda x: np.round(x[1]) if np.isnan(x[0]) else x[0], axis=1)
# 原価（unit_cost）の欠損値に中央値を補完する(axis=1で各行に適用)
df_product_4['unit_cost'] = df_product_4[['unit_cost', 'median_cost']]. \
                    apply(lambda x: np.round(x[1]) if np.isnan(x[0]) else x[0], axis=1)

# isnull().sum()で欠損を確認する
df_product_4.isnull().sum()

In [ ]:
# (解法2) 否v＞0.25なのでエラー
# 各商品の小区分（category_small_cd）ごとに中央値を出す
df_tmp = df_product.groupby('category_small_cd').agg(median_price=('unit_price', 'median'), 
                                                     median_cost=('unit_cost', 'median')).reset_index()
# 商品データフレーム（df_product）と中央値が入ったデータフレーム(df_tmp)を結合する
df_product_4 = pd.merge(df_product, df_tmp, how='inner', on='category_small_cd')
# maskを用いて欠損値(Nan)の部分はマスク（見えないように)して、中央値を代入する
# mask(隠す条件, 埋める値)、roundで四捨五入
df_product_4['unit_price'] = (df_product_4['unit_price'].mask(df_product_4['unit_price'].isnull(),
                                                             df_product_4['median_price'].round()))
df_product_4['unit_cost'] = (df_product_4['unit_cost'].mask(df_product_4['unit_cost'].isnull(),
                                                             df_product_4['median_price'].round()))

# isnull().sum()で欠損を確認する
df_product_4.isnull().sum()

In [ ]:
#  解３
# 商品データフレーム（df_product）をコピーする

df_product_4 = df_product.copy()
# 単価（unit_price）と原価（unit_cost）それぞれの処理を行う
# fillnaを用いて欠損値を補完する。
# 各商品の小区分（category_small_cd）ごとに中央値を出す。transformを用いる。
for x in ['unit_price', 'unit_cost']:
    df_product_4[x] = df_product_4[x].fillna(df_product_4.groupby('category_small_cd')[x].transform('median').round())
df_product_4.isnull().sum()


---
> P-084: 顧客データフレーム（df_customer）の全顧客に対し、全期間の売上金額に占める2019年売上金額の割合を計算せよ。ただし、販売実績のない場合は0として扱うこと。そして計算した割合が0超のものを抽出せよ。 結果は10件表示させれば良い。また、作成したデータにNAやNANが存在しないことを確認せよ。

In [ ]:
# 解１
df_tmp_1 = df_receipt.query('20190101 <= sales_ymd <= 20191231')
df_tmp_1 = pd.merge(df_customer['customer_id'],df_tmp_1[['customer_id','amount']],how='left',on='customer_id').\
groupby('customer_id').sum().reset_index().rename(columns={'amount':'amount_2019'})

df_tmp_2 = pd.merge(df_customer['customer_id'],df_receipt[['customer_id','amount']],how='left',on='customer_id').\
groupby('customer_id').sum().reset_index()

df_tmp = pd.merge(df_tmp_1,df_tmp_2,how='inner',on='customer_id')
df_tmp['amount_2019'] = df_tmp['amount_2019'].fillna(0)
df_tmp['amount'] = df_tmp['amount'].fillna(0)

df_tmp['amount_rate'] = df_tmp['amount_2019'] / df_tmp['amount']
df_tmp['amount_rate'] = df_tmp['amount_rate'].fillna(0)

df_tmp.query('amount_rate > 0').head(10)

In [ ]:
# 解２
df_tmp_1 = df_receipt.query('20190101 <= sales_ymd <= 20191231')
df_tmp_1 = pd.merge(df_customer['customer_id'],df_tmp_1[['customer_id','amount']],how='left',on='customer_id').\
groupby('customer_id').sum().reset_index().rename(columns={'amount':'amount_2019'})

df_tmp_2 = pd.merge(df_customer['customer_id'],df_receipt[['customer_id','amount']],how='left',on='customer_id').\
groupby('customer_id').sum().reset_index()

df_tmp = pd.merge(df_tmp_1,df_tmp_2,how='inner',on='customer_id')
df_tmp['amount_rate'] = df_tmp['amount_2019'] / df_tmp['amount']

In [ ]:
df_tmp.query('amount_rate > 0').head(10)

---
> P-085: 顧客データフレーム（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いて経度緯度変換用データフレーム（df_geocode）を紐付け、新たなdf_customer_1を作成せよ。ただし、複数紐づく場合は経度（longitude）、緯度（latitude）それぞれ平均を算出すること。


In [ ]:
df_geocode.head(5)

In [ ]:
df_geocode.duplicated().sum()

In [ ]:
df_customer.head(3)

In [ ]:
df_customer_1 = pd.merge(df_customer[['customer_id','postal_cd']],\
                       df_geocode[['postal_cd','longitude','latitude']],\
                         how='inner',on='postal_cd')
df_customer_1 = df_customer_1.groupby('customer_id').\
agg({'longitude':'mean','latitude':'mean'}).reset_index().\
rename(columns={'longitude':'m_longitude','latitude':'m_latitude'})

df_customer_1 = pd.merge(df_customer,df_customer_1,how='inner',on='customer_id')
df_customer_1.head(3)

---
> P-086: 前設問で作成した緯度経度つき顧客データフレーム（df_customer_1）に対し、申込み店舗コード（application_store_cd）をキーに店舗データフレーム（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客の緯度・経度を用いて距離（km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示すれば良い。

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [ ]:
#  mathを使ったパターン
def calc_distance(x1,y1,x2,y2):
    distance = 6371 * math.acos(math.sin(math.radians(y1))*math.sin(math.radians(y2))\
    +math.cos(math.radians(y1))*math.cos(math.radians(y2))\
    *math.cos(math.radians(x1)-math.radians(x2)))
    return distance

df_tmp = pd.merge(df_customer_1,df_store,how='inner',left_on='application_store_cd',right_on='store_cd')
df_tmp['distance'] = df_tmp[['m_longitude','m_latitude','longitude','latitude']].\
apply(lambda x : calc_distance(x[0],x[1],x[2],x[3]),axis=1)
df_tmp[['customer_id','address_x','address_y','distance']].head(10)

$$
緯度（ラジアン）：\phi \\
経度（ラジアン）：\lambda \\
距離L = 6371 * arccos(sin \phi_1 * sin \phi_2
+ cos \phi_1 * cos \phi_2 * cos(\lambda_1 − \lambda_2))
$$

In [ ]:
# 別解　計算式の関数を作成
def calc_distance(x1,y1,x2,y2):
    x1_r = np.radians(x1)
    y1_r = np.radians(y1)
    x2_r = np.radians(x2)
    y2_r = np.radians(y2)
    return 6371 * np.arccos(np.sin(y1_r)*np.sin(y2_r) \
                            + np.cos(y1_r)*np.cos(y2_r)\
                            *np.cos(x1_r - x2_r))
df_tmp = pd.merge(df_customer_1,df_store,how='inner',left_on=['application_store_cd'],right_on=['store_cd'])
df_tmp['distance']  = calc_distance(df_tmp['m_longitude'],df_tmp['m_latitude'],\
                                          df_tmp['longitude'],df_tmp['latitude'])
df_tmp[['customer_id','address_x','address_y','distance']].head(10)
    

---
> P-087:  顧客データフレーム（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなし、1顧客1レコードとなるように名寄せした名寄顧客データフレーム（df_customer_u）を作成せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残すものとし、売上金額合計が同一もしくは売上実績の無い顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
df_receipt.head(3)

In [ ]:
df_tmp = df_receipt.groupby('customer_id').agg({'amount':sum}).reset_index()
df_customer_u = pd.merge(df_customer,df_tmp,how='left',on='customer_id').\
sort_values(['amount','customer_id'],ascending=[False,True])
df_customer_u.drop_duplicates(subset=['customer_name','postal_cd'],keep='first',inplace=True)
print('現少数：',len(df_customer) - len(df_customer_u))

In [ ]:
df_customer_u.head(3)

---
> P-088: 前設問で作成したデータを元に、顧客データフレームに統合名寄IDを付与したデータフレーム（df_customer_n）を作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定

In [ ]:
df_customer_n = pd.merge(df_customer,df_customer_u[['customer_id','customer_name','postal_cd']],\
                        how='inner',on=['customer_name','postal_cd'])
# df_customer_n.head(3)
df_customer_n.rename(columns={'customer_id_x':'customer_id','customer_id_y':'integration_id'},inplace=True)
# df_customer_n.head(3)

print('ID数の差',len(df_customer_n['customer_id'].unique()) - len(df_customer_n['integration_id'].unique()))

---
> P-閑話: df_customer_1, df_customer_nは使わないので削除する。

In [ ]:
del df_customer_1
del df_customer_n

---
> P-089: 売上実績のある顧客に対し、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
df_tmp = pd.merge(df_customer,df_receipt['customer_id'],how='inner',on='customer_id')
df_train,df_test = train_test_split(df_tmp,test_size=0.2,random_state=71)
print('学習データ割合：',len(df_train) / len(df_tmp))
print('テストデータ割合：',len(df_test) / len(df_tmp))

---
> P-090: レシート明細データフレーム（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月のモデル構築用データを3セット作成せよ。

In [ ]:
# レシート明細データフレーム（df_receipt）をコピーする
df_tmp = df_receipt.copy()
# sales_ymdをint64からstr型へ変更し、スライスを用いて年月を取り出す
df_tmp['sales_ymd'] = df_tmp['sales_ymd'].astype('str').str[:6]

# sales_ymd(年月)でグループ分けして、売上金額（amount）を合計する
df_tmp = df_tmp.groupby('sales_ymd').agg({'amount':'sum'}).reset_index()

# 学習データとテストデータに分ける関数
def split_date(df,train_size,test_size,slide_window,start_point):
    train_start = start_point  * slide_window
    test_start = train_start + train_size
    return df[train_start : test_start] , df[test_start : test_start + test_size]

# モデル構築用データを3セット作成
df_train_1,df_test_1 = split_date(df_tmp , train_size = 12 , test_size = 6 , slide_window = 6 , start_point = 0)
df_train_2,df_test_2 = split_date(df_tmp , train_size = 12 , test_size = 6 , slide_window = 6 , start_point = 1)
df_train_3,df_test_3 = split_date(df_tmp , train_size = 12 , test_size = 6 , slide_window = 6 , start_point = 2)
            

In [ ]:
df_train_1

In [ ]:
df_test_1

In [ ]:
'''(補足: 問題を理解する)
学習用は１２ヶ月データでテスト用は６ヶ月データとするモデルは３つ

モデル 	学習データ 	学習データ範囲 	テストデータ 	テストデータ範囲
モデル1 	df_train_1 	2017/01月〜2017/12月 	df_train_1 	2017/01月〜2017/12月
モデル2 	df_train_2 	2017/06月〜2018/06月 	df_train_2 	2018/06月〜2018/12月
モデル3 	df_train_3 	2018/01月〜2018/12月 	df_train_3 	2019/01月〜2019/06月
'''

---
> P-091: 顧客データフレーム（df_customer）の各顧客に対し、売上実績のある顧客数と売上実績のない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
# レシート明細データフレーム（df_receipt）を顧客ごとにグループ分けし、売り上げを合計する
df_tmp = df_receipt.groupby('customer_id').agg({'amount':'sum'}).reset_index()
# 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）を結合(左結合)
df_tmp = pd.merge(df_customer,df_tmp,how='left',on='customer_id')
# 売り上げ実績(amount)がある場合は１、ない場合は０を入れる
df_tmp['buy_flg'] = df_tmp['amount'] . apply(lambda x : 0 if np.isnan(x) else 1)

# 売上実績のある顧客数と売上実績のない顧客数の件数を出力する
print('０の件数' , len(df_tmp.query('buy_flg==0')))
print('１の件数 ' ,len(df_tmp.query('buy_flg==1')))

positive_count = len(df_tmp.query('buy_flg == 1'))

# RandomUnderSampler でアンダーサンプリングする
rs = RandomUnderSampler(random_state = 71)
df_sample, _ = rs.fit_sample(df_tmp , df_tmp.buy_flg)

# 売上実績のある顧客数と売上実績のない顧客数の件数を出力する
print('０の件数', len(df_sample.query('buy_flg == 0')))
print('１の件数', len(df_sample.query('buy_flg == 1')))

---
> P-092: 顧客データフレーム（df_customer）では、性別に関する情報が非正規化の状態で保持されている。これを第三正規化せよ。

In [ ]:
# 顧客データフレーム（df_customer）のgender_cdの重複を取り除く
df_gender = df_customer[['gender_cd','gender']].drop_duplicates()

# 性別のデータフレーム(df_gender)を作成したので顧客データフレームからgenderカラムを削除する
df_customer_s = df_customer.drop(columns='gender')

In [ ]:
df_customer_s.head(3)

---
> P-093: 商品データフレーム（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータフレーム（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データフレームを作成せよ。

In [ ]:
df_product.head(1)

In [ ]:
df_category.head(3)

In [ ]:
df_product_full = pd.merge(df_product,df_category[['category_small_cd','category_major_name',\
                                                  'category_medium_name','category_small_name']],\
                                                  how='inner',on='category_small_cd')
df_product_full.head()
#キーを"category_small_cd"としているのは、"category_major_cd"や"category_medium_cd"ではカテゴリーとなっており、細かく分類できないからである。

---
> P-094: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
# to_csv(ファイル形式はCSV)を用いて出力する
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.csv',
                      encoding = 'utf_8_sig',
                      index = False)

---
> P-095: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはCP932

In [ ]:
#CP932とは？
Microsoft コードページ 932（CP932）は、マイクロソフト及び、MS-DOSのOEMベンダがShift_JISを独自に拡張した文字コードである。
df_product_full.to_csv('./data/P_df_product_full_CP932_header.csv',encoding='CP932',index=False)

---
> P-096: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_product_full.to_csv('./data/p_df_product_full_UTF-8_noh.csv',header='False',encoding='UTF-8',index=False)

---
> P-097: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp = pd.read_csv('./data/P_df_product_full_UTF-8_header.csv')
df_tmp.head(10)

---
> P-098: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はCSV（カンマ区切り）
> - ヘッダ無し
> - 文字コードはUTF-8

In [ ]:
df_tmp = pd.read_csv('./data/P_df_product_full_UTF-8_noh.csv',header=None)
df_tmp.head(10)

---
> P-099: 先に作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。なお、出力先のパスはdata配下とする。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_product_full.to_csv('./data/P_df_product_full_UTF-8_header.tsv',sep='\t',encoding='UTF-8',index=False)


---
> P-100: 先に作成した以下形式のファイルを読み込み、データフレームを作成せよ。また、先頭10件を表示させ、正しくとりまれていることを確認せよ。
>
> - ファイル形式はTSV（タブ区切り）
> - ヘッダ有り
> - 文字コードはUTF-8

In [ ]:
df_tmp = pd.read_table('./data/P_df_product_full_UTF-8_header.tsv',encoding='UTF-8')
df_tmp.head(10)

## 終了